# Elasticsearch를 사용하자!

In [65]:
#!pip install elasticsearch

In [8]:
from elasticsearch import Elasticsearch
#es = Elasticsearch()

In [9]:
import pprint

INDEX_NAME = 'wiki_docs'
INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer": "nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }

  }
}

In [75]:
import os
import json

with open('../data/wikipedia_documents.json') as data_file:
    local = json.load(data_file)

wiki_corpus = []
DOCS = {}

for i in range(len(local)):
    wiki_corpus.append(local[str(i)])
    
for i in range(len(wiki_corpus)): 
    data = {}
    DOCS[int(i)] = {'title' : wiki_corpus[i]['title'],
                    'content' : wiki_corpus[i]['text']}

In [10]:
try:
    es.transport.close()
except:
    pass

es = Elasticsearch(
    cloud_id="My_deployment:YXAtbm9ydGhlYXN0LTIuYXdzLmVsYXN0aWMtY2xvdWQuY29tOjkyNDMkYjUzNTg2NDllYzUwNDFkN2I5YjY2NzlhZTgyMDFkNmMkNWU1NjIyYzM2MDE0NDg4Yzk4MjNmZTI0MzQzYTBmYTc=",
    http_auth=("elastic", "IAfCPHmBlqHxGCfTgla8d0cR"),
)

In [11]:
es.info()

{'name': 'instance-0000000000',
 'cluster_name': 'b5358649ec5041d7b9b6679ae8201d6c',
 'cluster_uuid': 'zbbP7ubXRyOsqw2z4EnFRQ',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [12]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-12-b9190deb7ce1>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-12-b9190deb7ce1>:4: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_docs'}

In [80]:
import time
cnt = 0

for doc_id, doc in DOCS.items():
    if cnt % 1000 == 0:
        print('{}, time : {}'.format(cnt, time.time()))
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)
    time.sleep(0.1)
    cnt += 1
print(es)

0, time : 1634623106.1809251


<ipython-input-80-ea46b538349f>:7: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME,  id=doc_id, body=doc)


100, time : 1634623118.304454
200, time : 1634623130.1287959
300, time : 1634623142.0207546
400, time : 1634623153.8579273
500, time : 1634623165.662557
600, time : 1634623177.5276072
700, time : 1634623189.2796965
800, time : 1634623201.2591362
900, time : 1634623213.0696123
1000, time : 1634623224.8083644
1100, time : 1634623236.5673158
1200, time : 1634623248.3251915
1300, time : 1634623260.014034
1400, time : 1634623272.0211694
1500, time : 1634623283.8363967
1600, time : 1634623295.5467484
1700, time : 1634623307.2260714
1800, time : 1634623318.8726702
1900, time : 1634623330.5224051
2000, time : 1634623342.1320775
2100, time : 1634623353.721018
2200, time : 1634623365.6134582
2300, time : 1634623377.3134644
2400, time : 1634623389.0579386
2500, time : 1634623400.7437787
2600, time : 1634623412.4277267
2700, time : 1634623424.092559
2800, time : 1634623435.7603784
2900, time : 1634623447.382685
3000, time : 1634623459.185789
3100, time : 1634623470.8505044
3200, time : 1634623482.

In [82]:
doc = es.get(index=INDEX_NAME, id=2)
pprint.pprint(doc)

{'_id': '2',
 '_index': 'wiki_docs',
 '_primary_term': 1,
 '_seq_no': 145,
 '_source': {'content': '현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친일파인 아버지 '
                        '백낙승과 어머니 조종희 사이의 3남 2녀 중 막내로 태어났다. 그후 종로구 창신동 197번지 '
                        '소위 "큰대문집"에서 18세까지 살았다. 수송국민학교와 경기제1고등보통학교를 다니면서 피아니스트 '
                        '신재덕에게 피아노 연주를, 작이건우]]에게 작곡을 각각 배웠다. 이때 한국이 낳은 작곡가 '
                        '김순남을 사사했다. 1949년 그는 홍콩 로이덴 스쿨로 전학했으며, 한국 전쟁이 발발하기 이전 '
                        '가족이 일본으로 이주했다. 그 후 일본으로 건너가 1952년 도쿄 대학교 문과부에 입학했다. '
                        '2년 후 미술사학 및 미학으로 전공을 정했지만, 실제로는 일본 당대의 작곡가 모로이 사부로, '
                        '미학자 노무라 요시오 등에게서 작곡과, 음악사학을 공부했다. 졸업 논문은 ‘아르놀트 쇤베르크 '
                        '연구’이다.\n'
                        '\n'
                        '1956년 백남준은 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양의 '
                        '건축, 음악사, 철학 등을 공부하였다. 뮌헨 대학교 입학 1년 후에는 프라이부르크 국립 음악 '
                        '대학교로 옮겨 볼프강 포르트너 교수에게 배우지만, 곧 쇤베르크 이후 현대음악

In [83]:
tv = es.termvectors(index=INDEX_NAME, id=2,body={"fields" : ["content","title"]})

In [84]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'wiki_docs',
 '_type': '_doc',
 '_version': 2,
 'found': True,
 'term_vectors': {'content': {'field_statistics': {'doc_count': 60756,
                                                   'sum_doc_freq': 27183954,
                                                   'sum_ttf': 43119920},
                              'terms': {'1': {'term_freq': 7,
                                              'tokens': [{'end_offset': 141,
                                                          'position': 71,
                                                          'start_offset': 140},
                                                         {'end_offset': 513,
                                                          'position': 241,
                                                          'start_offset': 512},
                                                         {'end_offset': 3542,
                                                          'position': 1635,
             

In [85]:
query = "정여창은 어떻게 죽었나?"
res = es.indices.analyze(index=INDEX_NAME,
                                 body={
                                       "analyzer" : "korean",
                                        "text" : query
                                 }
                        )
pprint.pprint(res)

{'tokens': [{'end_offset': 2,
             'position': 0,
             'start_offset': 0,
             'token': '11',
             'type': 'word'},
            {'end_offset': 5,
             'position': 0,
             'positionLength': 2,
             'start_offset': 0,
             'token': '11 12',
             'type': 'shingle'},
            {'end_offset': 5,
             'position': 1,
             'start_offset': 3,
             'token': '12',
             'type': 'word'},
            {'end_offset': 7,
             'position': 1,
             'positionLength': 2,
             'start_offset': 3,
             'token': '12 세기',
             'type': 'shingle'},
            {'end_offset': 7,
             'position': 2,
             'start_offset': 5,
             'token': '세기',
             'type': 'word'},
            {'end_offset': 8,
             'position': 2,
             'positionLength': 2,
             'start_offset': 5,
             'token': '세기 에',
             'type': 'shin

In [150]:
res = es.search(index=INDEX_NAME, q=query, size=1)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '35393',
                    '_index': 'wiki_docs',
                    '_score': 34.118134,
                    '_source': {'content': '이 불상군은 1978년 발견되어 학계에 알려졌으며 봉황리 '
                                           '햇골산 중턱 두 곳에 약간 떨어져 위치한다. 아래편에는 앉은 '
                                           '자세의 본존불과 공양상, 반가사유상과 보살상 등 모두 8구로 '
                                           '이루어진 불상군이 위치한다. 불상들은 바위가 놓인 방향에 따라 '
                                           '동남쪽을 향하고 있으며 각각의 크기는 대략 1m 안팎이다. '
                                           '여래상의 당당한 어깨와 가슴, 손 모양, 두꺼운 옷주름 그리고 '
                                           '공양상의 고리장식과 허리띠 처리 등은 삼국시대 불상에서 보이는 '
                                           '특징이며, 특히 괭이모양의 대좌나 보살상의 갸름한 얼굴은 '
                                           '고구려 양식을 잘 따르고 있다. 왼편으로 조금 떨어진 윗편에는 '
                                           '규모가 큰 마애여래좌상이 동쪽을 향하여 조각되어 있는데 무릎에 '
          

## 확인해보기

1. json 파일을 불러와서 index와 비교한다.
2. train.csv 파일에서 query와 context 쌍을 가져온다.
3. elastic search에다가 query 하나씩 넣어두고 csv파일에 있는 데이터 수에서 얼마의 정확도를 가지는지 따져보자

In [4]:
import pandas as pd

train_file = '../../train.csv'
valid_file = '../../valid.csv'
train_data = pd.read_csv(train_file)
valid_data = pd.read_csv(valid_file)

data = pd.concat([train_data, valid_data])

In [5]:
import re

def pre_processing(query):
    Hangul = re.compile('[^ ㄱ-ㅣ가-힣|0-9|a-z|A-Z|<>]+')
    pre_query = Hangul.sub(' ', query)

    return pre_query

neg_sample = []

total_context = len(data)
correct = 0
similary_size = 1

for i in range(total_context):
    temp = 0
    
    query = pre_processing(data.iloc[i]['question'])
    context = data.iloc[i]['context']    
    predict_context = es.search(index=INDEX_NAME, q = query, size=similary_size)
    
    for i in range(similary_size):
        if DOCS[int(predict_context['hits']['hits'][i]['_id'])]['content'] == context:
            temp += 1
    
    if temp != 1:
        if len(neg_sample) != 10:
            negs = []
            for i in range(similary_size):
                negs.append(DOCS[int(predict_context['hits']['hits'][i]['_id'])]['content'])
            neg_sample.append((negs, context, query))
    else:
        correct += 1

print('Evaluation Done!')
#     print('predict : \n {}'.format(DOCS[int(predict_context['hits']['hits'][0]['_id'])]['content']))
#     print(f'-'*30)
#     print('target_context : \n {}'.format(context))
#     print('\n')

NameError: name 'es' is not defined

In [191]:
for i in range(10):
    print(f'-'*100)
    print('{}.query : {}\n'.format(i, neg_sample[i][2]))
    for j in range(len(neg_sample[i][0])):
        print('{}.search : {}\n'.format(j, neg_sample[i][0][j]))
    print('target: {}\n'.format(neg_sample[i][1]))
    print(f'-'*100)

----------------------------------------------------------------------------------------------------
0.query : 제2차 세계 대전에 참전하여 사망한 자식은 

0.search : 제1차 세계 대전(World War I, 약칭:WW1 등)은 1914년 7월 28일부터 1918년 11월 11일까지 일어난 유럽을 중심으로 한 세계 대전이다. 제2차 세계 대전이 발생하기 전까지는 단순히 세계 대전(World War) 또는 대전(Great War)이라고 불렸다.    미국에서는 처음에 유럽 전쟁(European War)라고 불렸다.  제1차 세계 대전으로 병사 900만명 이상이 사망했다. 기술 및 산업의 고도화와 전술적 교착 상태로 인해 사상자 비율이 악화되었다. 제1차 세계 대전은 사망자가 가장 많았던 전쟁 중 하나이며, 참전국의 수많은 혁명 등을 포함하여 주요한 정치적 변화가 일어났다. 

이 전쟁은 전 세계의 경제를 두 편으로 나누는 거대한 강대국들 동맹끼리의 충돌이다.  한쪽 편은 대영제국, 프랑스, 러시아 제국의 삼국 협상을 기반으로 한 협상국이며, 다른 한편은 독일 제국과 오스트리아-헝가리 제국이 있는 동맹국이다. 이탈리아 왕국은 독일 제국, 오스트리아-헝가리 제국과 함께 삼국 동맹에 가입되어 있었지만 동맹국에 참여하지 않았고 나중에는 협상국으로 참가하며 오스트리아-헝가리 제국을 침공했다.  이러한 동맹은 재조직되었고 더 많은 국가가 전쟁에 참여하도록 압력을 가하면서 확장되었다. 이탈리아 왕국, 일본 제국, 미국이 연합국에 가입했으며 오스만 제국, 불가리아 왕국이 동맹국에 가담했다. 궁극적으로 유럽인 6천만 명을 포함한 군인 7천만 명이 전쟁에 가담하면서 역사적으로 가장 큰 전쟁 중 하나에 동원되었다.  

이 전쟁의 근본적인 원인은 신제국주의때문이었지만, 직접적인 원인은 1914년 6월 28일 사라예보에서 오스트리아-헝가리 제국 왕위 후계자인 프란츠 페르디난트 대공이 세르

In [192]:
print('acc : {}'.format((correct/total_context)*100))

acc : 87.76240458015268
